In [39]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy import sparse
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any, Literal
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

In [42]:
df = pd.read_csv("res\\rating.csv")

In [43]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df

In [44]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

  0%|          | 0/100 [02:44<?, ?it/s]


In [45]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [46]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [47]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


In [53]:
MappingIntToObject = Mapping[int, Tuple[Any, Literal['source', 'target']]]
        
# https://dspace.mit.edu/bitstream/handle/1721.1/86737/49317146-MIT.pdf?sequence=2&isAllowed=y
class HITS():
    
    def __init__(self, data:pd.DataFrame, source: str, target:str, edge_attr:str, delta: float,
                 personalize_by: Any, max_iter: int = 100) -> None:
        
        self.delta = delta
        self.target = target
        self.source = source
        self.max_iter = max_iter
        
        (self.__transition_matrix,
         self.__N_to_obj,
         self.__obj_source_to_N) = self.__create_transition_matrix_from_pd(data, source,
                                                             target, edge_attr)
        self.__personalize_index = self.__obj_source_to_N[personalize_by]
         
        
    def __create_transition_matrix_from_pd(self, data: pd.DataFrame,
               source: str, target: str, edge_attr: str) -> Tuple[sparse.csr_matrix, MappingIntToObject]:
    
        obj_source_to_N = {}
        obj_target_to_N = {}
        N_to_obj = {}
        
        source_uniq = set(data[source])
        for index, obj in zip(range(len(source_uniq)), source_uniq):
            obj_source_to_N[obj] = index
            N_to_obj[index] = (obj, 'source')
            
        
        target_uniq = set(data[target])
        for index, obj in zip(range(len(source_uniq), len(source_uniq) + len(target_uniq)), target_uniq):
            obj_target_to_N[obj] = index
            N_to_obj[index] = (obj, 'target')
        
        count_top = len(N_to_obj)
        transition_matrix = sparse.lil_matrix((count_top, count_top))
        
        # Функция для отображения значений согласно словарю
        def map_source_to_N(value):
            return obj_source_to_N[value]

        def map_target_to_N(value):
            return obj_target_to_N[value]

        # Создаем векторизованную версию функций
        vectorized_source_to_N = np.vectorize(map_source_to_N)
        vectorized_target_to_N = np.vectorize(map_target_to_N)

        # Применяем векторизованные функции к массивам
        start_nodes = vectorized_source_to_N(data[source].to_numpy())  # Начальные вершины ребер
        end_nodes = vectorized_target_to_N(data[target].to_numpy())    # Конечные вершины ребер
        weights = np.array(data[edge_attr])
        
        # Заполнение матрицы значениями из массивов
        transition_matrix[start_nodes, end_nodes] = weights

        # Если граф неориентированный, добавьте обратные ребра
        transition_matrix[end_nodes, start_nodes] = weights
        
        # Получаем суммы элементов каждой строки в виде numpy массива
        
        transition_matrix_csr = sparse.csr_matrix(transition_matrix)
        del transition_matrix
        
        row_sums = transition_matrix_csr.sum(axis=1)
        transition_matrix_csr /= row_sums    

        return (transition_matrix_csr,
                N_to_obj, obj_source_to_N)
    
    def get_rank(self) -> pd.DataFrame:
        
        authoritativeness, hubness = self.__get_rank(self.__transition_matrix,
                                       self.max_iter)
         
         
        target_column_auth = []
        rank_column_auth = []
        for ind, el_rank in enumerate(authoritativeness):
            if self.__N_to_obj[ind][1] == 'target':
                target_column_auth.append(self.__N_to_obj[ind][0])
                rank_column_auth.append(el_rank)
                
        target_column_hub = []
        rank_column_hub = []
        for ind, el_rank in enumerate(hubness):
            if self.__N_to_obj[ind][1] == 'target':
                target_column_hub.append(self.__N_to_obj[ind][0])
                rank_column_hub.append(el_rank)

        return pd.DataFrame({self.target: target_column_auth ,
                             'authoritativeness': rank_column_auth,
                             'hubness': rank_column_hub})
    
    def __get_rank(self, transition_matrix: sparse.csr_matrix,
                       max_iter: int = 200) -> np.ndarray:
        

        transition_T = transition_matrix.transpose()
        height = transition_matrix.shape[0]
        user_rank_vec = transition_matrix.getrow(self.__personalize_index).T#sparse.csr_matrix(np.ones((height,1)) / height)
        zeros = np.zeros(height)
        zeros[self.__personalize_index] += self.delta
        sup_vec = sparse.csr_matrix(zeros).T
        hub = user_rank_vec.copy()
        auth = user_rank_vec.copy()

        # Инициализация прогресс-бара
        progress_bar = tqdm(total=max_iter)
        # Update the PageRank vector until convergence! Our convergence criterion is to see whether
        # the magnitude of the difference of the PageRank vector after an update is 0
        for i in range(max_iter):
            # (deep) copy the current PageRank vector to compare with the updated vector
            
            old_hub = hub.copy()
            old_auth = auth.copy()
            
            old_hub += sup_vec
            old_auth += sup_vec
            
            old_hub /= old_hub.sum()
            old_auth /= old_auth.sum()
            
            # Update!
            hub = sparse.csr_matrix(transition_matrix.multiply(old_auth.T).sum(axis=1))
            auth = sparse.csr_matrix(transition_T.multiply(old_hub.T).sum(axis=1))
                                
            if i % 10 == 0:
                hub /= hub.sum()
                auth /= auth.sum()
            
            magnitude = norm(hub - old_hub)
            progress_bar.set_description("magnitude: {:.6f}".format(magnitude))
            progress_bar.update()
            
            if magnitude < 10**-8:
                break
            
        progress_bar.close()
        return auth.toarray().flatten(), hub.toarray().flatten()

  
data_test = pd.DataFrame({
    'source': ['A', 'B', 'B', 'B'],
    'target': ['X', 'X', 'Z', 'C'],
    'edge_attr': [4, 4, 5, 3]
})  
    
pg = HITS(data_test, 'source', 'target', 'edge_attr', 0.01, 'A', 100)
pg.get_rank()

magnitude: 0.741595: 100%|██████████| 100/100 [00:01<00:00, 85.08it/s]


,target,authoritativeness,hubness
0,X,0.009061,0.495222
1,C,0.000387,0.872128
2,Z,0.000645,0.872128


In [109]:
hits = HITS(data=data, source='userId', target='movieId', edge_attr='rating',
          delta=0.2, personalize_by=1, max_iter=50)

In [110]:
prediction_movie_for_user1 = hits.get_rank()

magnitude: 7.111802: 100%|██████████| 50/50 [02:03<00:00,  2.48s/it]


In [111]:
print(prediction_movie_for_user1.sort_values(by='hubness', ascending=False).head())

       movieId  authoritativeness   hubness
24951   118942       2.034282e-10  0.377452
17350    87354       4.068564e-10  0.377452
23265   111126       3.559993e-10  0.377452
17358    87378       4.068564e-10  0.377452
17371    87420       4.068564e-10  0.377452


In [112]:
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~(prediction_movie_for_user1['movieId'].isin(train1['movieId']))]
print(len(prediction_movie_for_user1_without_famous))
prediction_movie_for_user1_without_famous.head()

26657


,movieId,authoritativeness,hubness
0,131072,9.472566e-10,0.000007
1,1,6.170056e-03,0.000006
2,2,2.005399e-03,0.000003
3,3,1.287147e-03,0.000002
4,4,1.698813e-04,0.000007


In [113]:
pred = prediction_movie_for_user1_without_famous.sort_values(by='authoritativeness', ascending=False).head(len(test1))
print(len(pred))
pred.head()

88


,movieId,authoritativeness,hubness
295,296,0.013585,0.000008
317,318,0.013247,0.000008
354,356,0.012471,0.000007
150,150,0.012172,0.000008
455,457,0.011967,0.000010


In [114]:
def accuracy_without_order(true_labels, predicted_labels):
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set)
    
    return accuracy

In [115]:
print(accuracy_without_order(test1['movieId'], pred['movieId']))

0.14772727272727273


In [116]:
movie_df = pd.read_csv('res/movie.csv')

In [117]:
prediction_moviename = pd.merge(movie_df, pred, on='movieId', how='inner')
prediction_moviename.sort_values(by='authoritativeness', ascending=False).head()

,movieId,title,genres,authoritativeness,hubness
34,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0.013585,0.000008
38,318,"Shawshank Redemption, The (1994)",Crime|Drama,0.013247,0.000008
44,356,Forrest Gump (1994),Comedy|Drama|Romance|War,0.012471,0.000007
19,150,Apollo 13 (1995),Adventure|Drama|IMAX,0.012172,0.000008
54,457,"Fugitive, The (1993)",Thriller,0.011967,0.000010


In [118]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
print(len(test_moviename))
test_moviename.sort_values(by='rating', ascending=False).head()

88


,movieId,title,genres,userId,rating,timestamp
73,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,1,5.0,2005-04-02 23:30:19
87,8636,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,1,4.5,2005-04-02 23:44:53
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
54,3499,Misery (1990),Drama|Horror|Thriller,1,4.0,2005-04-02 23:35:18


In [119]:
test_moviename[test_moviename['movieId'].isin(prediction_moviename['movieId'])]

,movieId,title,genres,userId,rating,timestamp
0,2,Jumanji (1995),Adventure|Children|Fantasy,1,3.5,2005-04-02 23:53:47
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,1,3.5,2005-04-02 23:33:39
2,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,3.5,2005-04-02 23:29:40
4,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1,4.0,2005-04-02 23:33:46
5,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,1,4.0,2005-04-02 23:31:43
6,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1,4.0,2005-04-02 23:32:47
7,318,"Shawshank Redemption, The (1994)",Crime|Drama,1,4.0,2005-04-02 23:33:18
9,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,1,3.5,2005-04-02 23:45:57
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24


In [120]:
from typing import Any, List

class O:
    
    def __init__(self, delta: Any) -> None:
        self.delta = delta
        
    def __call__(self, true_score_value: Any) -> int:
        return int(true_score_value > self.delta)
    
class T():
    def __init__(self, delta: Any) -> None:
        self.delta = delta
        
    def __call__(self, pred_value: Any) -> int:
        return int(pred_value > self.delta)

class I():
    
    def __init__(self, pagerankT:T, valuesO: O) -> None:
        self.pagerankT = pagerankT
        self.valuesO = valuesO
        
    def __call__(self, pred_p: Any, true_p: Any,
                 pred_q: Any, true_q: Any) -> int:
        if pred_p >= pred_q and \
            self.valuesO(true_p) < self.valuesO(true_q):
            return 1
        if pred_p <= pred_q and \
            self.valuesO(true_p) > self.valuesO(true_q):
            return 1
        return 0
    
    
class Pairord():
    
    def __init__(self, i: I) -> None:
        self.i = i
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        metric = len(pred_pagerank) * len(true_score)
        for p in range(len(pred_pagerank)):
            for q in range(len(true_score)):
                metric -= self.i(pred_pagerank[p], true_score[q],
                                        pred_pagerank[q], true_score[p])
            
        metric /= (len(pred_pagerank) * len(true_score))
        
        return metric
    
class Prec():
    
    def __init__(self, o: O, t: T) -> None:
        self.o = o
        self.t = t       
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        numerator = 0
        denominator = 0
        for p in range(len(pred_pagerank)):
            numerator += self.t(pred_pagerank[p]) * self.o(true_score[p])
            denominator += self.t(pred_pagerank[p])
        
        return numerator / denominator
    
    
class Rec():
    
    def __init__(self, o: O, t: T) -> None:
        self.o = o
        self.t = t       
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        numerator = 0
        denominator = 0
        for p in range(len(pred_pagerank)):
            numerator += self.t(pred_pagerank[p]) * self.o(true_score[p])
            denominator += self.o(true_score[p])
        
        return numerator / denominator
    
class F1():
    
    def __init__(self, prec: Prec, rec: Rec) -> None:
        self.prec = prec
        self.rec = rec
        
    def __call__(self, pred_pagerank: List[Any], true_score: List[Any]) -> float:
        p = self.prec(pred_pagerank, true_score)
        r = self.rec(pred_pagerank, true_score)
            
        return 2*p*r/(p + r)

In [121]:

for_score = prediction_movie_for_user1.merge(test1, how='inner', on='movieId')
o = O(test1['rating'].mean())
t = T(prediction_movie_for_user1['authoritativeness'].mean())
i = I(t, o)
pairroid = Pairord(i)
pred = np.array(for_score['authoritativeness'])
true = np.array(for_score['rating'])
pred_map = np.vectorize(lambda x: t(x))
true_map = np.vectorize(lambda x: o(x))

pred_trasformed = pred_map(pred)
true_trasformed = true_map(true)

In [122]:
# Вычисляем метрики
accuracy = accuracy_score(true_trasformed, pred_trasformed)
precision = precision_score(true_trasformed, pred_trasformed)
recall = recall_score(true_trasformed, pred_trasformed)
f1 = f1_score(true_trasformed, pred_trasformed)

# Выводим результаты
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Pairorder:", pairroid(for_score['authoritativeness'], for_score['rating']))

Accuracy: 0.5681818181818182
Precision: 0.5492957746478874
Recall: 0.8666666666666667
F1 Score: 0.6724137931034483
Pairorder: 0.6712293388429752
